# Observed UHI Magnitudes

## 1 - Import Necessary Modules

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

# warnings
import warnings
warnings.filterwarnings('ignore')

## 2 - Open Datasets

In [2]:
ds = pd.read_excel(r'ankara_0918T.xlsx', na_values=-999.0)
MODIS_terra_data = xr.open_dataset(r'Ankara_MODIS_Terra_1018.nc')
MODIS_aqua_data = xr.open_dataset(r'Ankara_MODIS_Aqua_1018.nc')

## 3 - Define Station Information

In [3]:
stations_selected = {'17127' : [837, 40.0788, 32.5657, 'Akıncı Airport'],
                     '17128' : [958, 40.124, 32.9992, 'Esenboğa Airport'],
                     '17129' : [807, 39.9558, 32.6854, 'Etimesgut Airport'],
                     '17130': [890, 39.9727, 32.8637, 'Ankara Bölge'],
                     '17131' : [819, 39.9343, 32.7387, 'Güvercinlik Airport'],
                     '17134': [1098, 39.8032, 32.8434, 'Ufuk Danışment'],
                     '17137': [1797, 39.7985, 32.9716, 'Elmadağ Radar'],
                     '17651' : [940, 39.1525, 32.1283, 'Polatlı Tigem'],
                     '17664' : [1030, 40.4729, 32.6441, 'Kızılcahamam'],
                     '17679' : [648, 40.1733, 31.332, 'Nallıhan'],
                     '17680' : [680, 40.1608, 31.9172, 'Beypazarı'],
                     '17715' : [1113, 39.92, 33.2125, 'Elmadağ Barutsan Fabrikası'],
                     '17728' : [886, 39.5834, 32.1624, 'Polatlı'],
                     '17729' : [1248, 39.5546, 33.1089, 'Bala'],
                     '17731' : [911, 38.9539, 33.4218, 'Şereflikoçhisar'],
                     '17733' : [1065, 39.613, 32.672, 'Haymana Tarım']}

urban1 = stations_selected['17130'] #ankara bölge
urban2 = stations_selected['17131'] #güvercinlik airport
rural1 = stations_selected['17128'] #esenboğa airport
rural2 = stations_selected['17134'] #ufuk danışment - gölbaşı

## 4 - Cancel Out Altitude Differences

In [4]:
#cancel out altitude differences using a universal lapse rate
dst = ds.copy(deep=True)
for i in stations_selected:
    factor = (stations_selected[i][0] * 0.65) / 100 # cancel out altitude differences
    dst[int(i)] = dst[int(i)] + factor

## 5 - Set Dates

In [9]:
date_used = pd.date_range('2009-01-01 00', '2018-12-31 23', freq='1H')
dst.index = date_used

In [45]:
spring_date_used = pd.date_range('2014-05-04 00', '2014-05-05 23', freq='1H')
#spring_dst.index = spring_date_used

summer_date_used = pd.date_range('2016-08-27 00', '2016-08-28 23', freq='1H')
#summer_dst.index = summer_date_used

In [46]:
spring_dst = dst.loc[spring_date_used]
summer_dst = dst.loc[summer_date_used]

## 6 - Calculate UHI

In [47]:
spring_dst.rename(columns={17130: 'urban1', 17131:'urban2',
                           17128: 'rural1', 17134:'rural2'},
                  inplace=True)

summer_dst.rename(columns={17130: 'urban1', 17131:'urban2',
                           17128: 'rural1', 17134:'rural2'},
                  inplace=True)

In [48]:
spring_UN = spring_dst[['urban1', 'urban2', 'rural1', 'rural2']]
summer_UN = summer_dst[['urban1', 'urban2', 'rural1', 'rural2']]

In [49]:
spring_UHI = pd.DataFrame()
spring_UHI['urban1-rural1'] = (spring_UN['urban1'] - spring_UN['rural1']).values
spring_UHI['urban1-rural2'] = (spring_UN['urban1'] - spring_UN['rural2']).values
spring_UHI['urban2-rural1'] = (spring_UN['urban2'] - spring_UN['rural1']).values
spring_UHI['urban2-rural2'] = (spring_UN['urban2'] - spring_UN['rural2']).values
spring_UHI.index = spring_date_used

In [50]:
summer_UHI = pd.DataFrame()
summer_UHI['urban1-rural1'] = (summer_UN['urban1'] - summer_UN['rural1']).values
summer_UHI['urban1-rural2'] = (summer_UN['urban1'] - summer_UN['rural2']).values
summer_UHI['urban2-rural1'] = (summer_UN['urban2'] - summer_UN['rural1']).values
summer_UHI['urban2-rural2'] = (summer_UN['urban2'] - summer_UN['rural2']).values
summer_UHI.index = summer_date_used

In [51]:
np.argmax(summer_UHI.max(axis=1))

17

In [59]:
summer_UHI.loc['2016-08-28 09:00:00':'2016-08-28 12:00:00'].mean().mean()

0.8722500000000006

In [53]:
np.argmax(spring_UHI.max(axis=1))

4

In [60]:
spring_UHI.loc['2014-05-05 09:00:00':'2014-05-05 12:00:00'].mean().mean()

0.034749999999998504